In [ ]:
#default_exp phenotypes.__init__

# Functions to extract phenotype info

> API details.

In [ ]:
#export
from combinatorial_gwas.data_catalog import get_catalog, get_parameters
import combinatorial_gwas
from pathlib import Path
import pandas as pd
from dataclasses import dataclass
from functools import partial
import numpy as np
from typing import List, Union
from fastcore.utils import partialler
import logging

In [ ]:
#export

@pd.api.extensions.register_dataframe_accessor("pheno")
@dataclass
class QueryDataframe():
    df: pd.DataFrame
    
    def query(self, **column_dict:dict):
        query_str = " and ".join([f"({col} {cond})" for col, cond in column_dict.items()])
        return self.df.query(query_str)

In [ ]:
#export
parameters = get_parameters()
parameters

{'example_test_data_ratio': 0.2,
 'example_num_train_iter': 10000,
 'example_learning_rate': 0.01,
 'template_gwas_result_file_link': 'https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/{phenotype_code}.gwas.imputed_v3.both_sexes.tsv.bgz',
 'genetic_file_path_template': '/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr{chrom_number}_v3.bgen',
 'sample_file_template': '/lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr21_v3_s487275.sample',
 'data_dict_file': '/lab/corradin_biobank/Phenotypes//Data_Dictionary_Showcase.csv',
 'pheno_file': '/lab/corradin_biobank/samples//neale_gwas_both_sexes_parquet/',
 'coding_file_path_template': '/lab/corradin_biobank/Phenotypes//data_codes/datacode-*.tsv',
 'id_col': 'f.eid'}

In [ ]:
#export
catalog_all = get_catalog()
catalog_all = catalog_all.reload()
catalog_all.list()

['example_iris_data',
 'globals_test',
 'disease_summary',
 'sample_ids',
 'chr_22',
 'ICD10_pheno_matrix',
 'phenotypes_info_Neale',
 'GWAS_results_links_df',
 'heritability_trait_level_summary',
 'heritability_trait_level_partitioned_z4_sig']

In [ ]:
sample_ids = catalog_all.load("sample_ids")

In [ ]:
#export
read_csv_compressed= partialler(pd.read_csv, sep="\t", compression= "gzip")
get_GWAS_result_link = partialler(parameters['template_gwas_result_file_link'].format)


---

### Find high heritability traits

In [ ]:
#export
heritability_Neale = catalog_all.load("heritability_trait_level_summary")
heritability_Neale.head()

/lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/lib/python3.8/site-packages/kedro/extras/datasets/pandas/csv_dataset.py:158: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(fs_file, **self._load_args)


,phenotype,description,h2_liability,h2_liability_se,h2_observed,h2_observed_se,h2_z,h2_p,h2_sig,confidence,...,sex,isNotPrimary,isBadPower,isLowNeff,isMidNeff,isExtremeSE,isHighSE,isSexBias,isBadOrdinal,isNumericOrdinal
0,100001_irnt,Food weight,0.068818,0.016857,0.068818,0.016857,4.082528,0.000022,z4,high,...,both_sexes,False,False,False,False,False,False,False,False,False
1,100002_irnt,Energy,0.064783,0.015784,0.064783,0.015784,4.104249,0.000020,z4,high,...,both_sexes,False,False,False,False,False,False,False,False,False
2,100003_irnt,Protein,0.034543,0.014619,0.034543,0.014619,2.362892,0.009066,nominal,high,...,both_sexes,False,False,False,False,False,False,False,False,False
3,100004_irnt,Fat,0.055197,0.018265,0.055197,0.018265,3.021937,0.001256,nominal,high,...,both_sexes,False,False,False,False,False,False,False,False,False
4,100005_irnt,Carbohydrate,0.051744,0.015357,0.051744,0.015357,3.369359,0.000377,nominal,high,...,both_sexes,False,False,False,False,False,False,False,False,False


In [ ]:
heritability_Neale.columns

Index(['phenotype', 'description', 'h2_liability', 'h2_liability_se',
       'h2_observed', 'h2_observed_se', 'h2_z', 'h2_p', 'h2_sig', 'confidence',
       'notes', 'intercept', 'intercept_se', 'intercept_z', 'intercept_p',
       'lambdaGC', 'mean_chi2', 'ratio', 'ratio_se', 'n', 'Neff',
       'variable_type', 'isBinary', 'n_cases', 'n_controls', 'prevalence',
       'source', 'sex', 'isNotPrimary', 'isBadPower', 'isLowNeff', 'isMidNeff',
       'isExtremeSE', 'isHighSE', 'isSexBias', 'isBadOrdinal',
       'isNumericOrdinal'],
      dtype='object')

In [ ]:
#export
display_cols = ['description', 'h2_liability', 'h2_sig', 'confidence', 'n_cases', 'n_controls', 'prevalence']

In [ ]:
heritability_Neale.query("h2_sig == ['z7', 'z4']").sort_values("h2_liability", ascending = False).head(60)

,phenotype,description,h2_liability,h2_liability_se,h2_observed,h2_observed_se,h2_z,h2_p,h2_sig,confidence,...,sex,isNotPrimary,isBadPower,isLowNeff,isMidNeff,isExtremeSE,isHighSE,isSexBias,isBadOrdinal,isNumericOrdinal
2543,50_irnt,Standing height,0.485223,0.021812,0.485223,0.021812,22.246180,6.140928e-110,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
2554,5097_irnt,6mm weak meridian (left),0.433908,0.038336,0.433908,0.038336,11.318693,5.302052e-30,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
1293,20153_irnt,"Forced expiratory volume in 1-second (FEV1), p...",0.433270,0.023709,0.433270,0.023709,18.274839,6.563795e-75,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
2579,5134_irnt,6mm strong meridian (left),0.427685,0.038038,0.427685,0.038038,11.243581,1.245346e-29,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
4155,2395_4,Hair/balding pattern: Pattern 4,0.423410,0.042975,0.198793,0.020177,9.852372,3.347251e-23,z7,high,...,male,False,False,False,False,False,False,False,False,False
2578,5133_irnt,6mm strong meridian (right),0.423025,0.036995,0.423025,0.036995,11.434813,1.400691e-30,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
2555,5098_irnt,6mm weak meridian (right),0.418781,0.037019,0.418781,0.037019,11.312652,5.680164e-30,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
2302,30100_irnt,Mean platelet (thrombocyte) volume,0.406220,0.057866,0.406220,0.057866,7.019956,1.109691e-12,z7,medium,...,both_sexes,False,False,False,False,False,True,False,False,False
2553,5096_irnt,3mm weak meridian (left),0.393818,0.038762,0.393818,0.038762,10.160001,1.495374e-24,z7,high,...,both_sexes,False,False,False,False,False,False,False,False,False
4152,2395_1,Hair/balding pattern: Pattern 1,0.387338,0.036491,0.227732,0.021454,10.614754,1.272325e-26,z7,high,...,male,False,False,False,False,False,False,False,False,False


In [ ]:
#export
quality_heritability_phenos = heritability_Neale.pheno.query(h2_sig = "in ['z7', 'z4']", source= " == 'icd10'", confidence= "in ['medium', 'high']").sort_values("h2_liability", ascending = False)
quality_heritability_phenos = quality_heritability_phenos.set_index("phenotype")
quality_heritability_phenos.head()[display_cols]

,description,h2_liability,h2_sig,confidence,n_cases,n_controls,prevalence
phenotype,,,,,,,
I83,Diagnoses - main ICD10: I83 Varicose veins of ...,0.189299,z7,medium,8763.0,352431.0,0.024261
I25,Diagnoses - main ICD10: I25 Chronic ischaemic ...,0.163823,z7,medium,12769.0,348425.0,0.035352
G56,Diagnoses - main ICD10: G56 Mononeuropathies o...,0.157519,z7,medium,8130.0,353064.0,0.022509
N81,Diagnoses - main ICD10: N81 Female genital pro...,0.152223,z4,medium,7511.0,186663.0,0.038682
M16,Diagnoses - main ICD10: M16 Coxarthrosis [arth...,0.151297,z4,medium,9136.0,352058.0,0.025294


In [ ]:
quality_heritability_phenos[display_cols]

,description,h2_liability,h2_sig,confidence,n_cases,n_controls,prevalence
phenotype,,,,,,,
I83,Diagnoses - main ICD10: I83 Varicose veins of ...,0.189299,z7,medium,8763.0,352431.0,0.024261
I25,Diagnoses - main ICD10: I25 Chronic ischaemic ...,0.163823,z7,medium,12769.0,348425.0,0.035352
G56,Diagnoses - main ICD10: G56 Mononeuropathies o...,0.157519,z7,medium,8130.0,353064.0,0.022509
N81,Diagnoses - main ICD10: N81 Female genital pro...,0.152223,z4,medium,7511.0,186663.0,0.038682
M16,Diagnoses - main ICD10: M16 Coxarthrosis [arth...,0.151297,z4,medium,9136.0,352058.0,0.025294
M17,Diagnoses - main ICD10: M17 Gonarthrosis [arth...,0.144360,z7,high,11497.0,349697.0,0.031831
I48,Diagnoses - main ICD10: I48 Atrial fibrillatio...,0.144326,z4,medium,6356.0,354838.0,0.017597
C44,Diagnoses - main ICD10: C44 Other malignant ne...,0.141495,z4,medium,9086.0,352108.0,0.025155
K80,Diagnoses - main ICD10: K80 Cholelithiasis,0.137236,z4,high,10520.0,350674.0,0.029126


In [ ]:
quality_heritability_phenos.index

Index(['I83', 'I25', 'G56', 'N81', 'M16', 'M17', 'I48', 'C44', 'K80', 'I21',
       'I20', 'K57', 'K40', 'D12', 'M54', 'C50', 'K22', 'R07', 'M20', 'H26',
       'K21', 'K44', 'M23', 'N39', 'R10', 'R31', 'I84'],
      dtype='object', name='phenotype')

In [ ]:
quality_heritability_phenos.shape

In [ ]:
pheno = quality_heritability_phenos.loc["I84"]
pheno

description         Diagnoses - main ICD10: I84 Haemorrhoids
h2_liability                                        0.057475
h2_liability_se                                     0.013482
h2_observed                                         0.009864
h2_observed_se                                      0.002314
h2_z                                                4.263044
h2_p                                                 0.00001
h2_sig                                                    z4
confidence                                              high
notes                                                    NaN
intercept                                           1.018834
intercept_se                                        0.008843
intercept_z                                         2.129842
intercept_p                                         0.016592
lambdaGC                                            1.074945
mean_chi2                                           1.085477
ratio                   

---

### Output phenotype matrix

In [ ]:
#export
logging.warning("Loading ICD phenotype matrix, this might take a while")
icd10_pheno_matrix = catalog_all.load("ICD10_pheno_matrix")


#get the first 3 character of ICD code
icd10_primary_cols = icd10_pheno_matrix.columns[icd10_pheno_matrix.columns.str.contains("primary")]
icd10_pheno_matrix = icd10_pheno_matrix.astype(str).apply(lambda x: x.str.slice(0,3))

logging.warning("Finished loading ICD10 matrix")

In [ ]:
#export

def upsample_pheno(pheno_df, balance_pheno, max_samples, random_state):
    weights = pheno_df[balance_pheno].replace(pheno_df[balance_pheno].value_counts(normalize=True).to_dict())
    pheno_df_upsampled = pheno_df.sample(max_samples, replace=True, weights = 1/weights, random_state=random_state)
    return pheno_df_upsampled


def get_phenotype(icd10_codes: Union[str, List[str]] ="I84", samples:np.array=None, max_samples:int = None, balance_pheno: str = None, random_state=42):
    """
    if samples argument is provided from genetic file, then find common set of samples and output ordered phenotype
    
    if `max_samples` is provided, then over-sample the data so that we have `max_samples/2` for both cases and controls
    """
    icd10_codes = [icd10_codes] if not isinstance(icd10_codes, list) else icd10_codes
    pheno_df_list = [icd10_pheno_matrix[icd10_primary_cols].isin([icd10_code]).any(axis=1).astype(int) for icd10_code in icd10_codes]
    pheno_df = pd.concat(pheno_df_list, axis=1)
    pheno_df.columns = icd10_codes
    
    if samples is not None:
        geno_pheno_sample_index_mask = np.isin(samples.astype(int), pheno_df.index)
        pheno_geno_samples_common_set = samples[geno_pheno_sample_index_mask].astype(int)
        pheno_df_ordered = pheno_df.loc[list(pheno_geno_samples_common_set), :]
        pheno_df_ordered = pheno_df_ordered.loc[~pheno_df_ordered.index.duplicated(keep="first"),:]
        sample_index = np.argwhere(geno_pheno_sample_index_mask).reshape(-1)
        
        if max_samples is not None:
            if balance_pheno is None:
                raise ValueError("Need to specify `balance_pheno` param: phenotype to balance during subsampling")
            pheno_df_ordered = upsample_pheno(pheno_df_ordered, balance_pheno, max_samples, random_state)
            
            sorter = np.argsort(samples.astype(int))
            sample_index = sorter[np.searchsorted(samples.astype(int), pheno_df_ordered.index, sorter=sorter)]
        
        assert np.allclose(samples[sample_index].astype(int), pheno_df_ordered.index), "sample mismatch between genotype and phenotype, something wrong with the `get_phenotype` function!"
        
        pheno_df_ordered.index = pheno_df_ordered.index.astype(str)
        return pheno_df_ordered
    return pheno_df

In [ ]:
test_one_pheno = get_phenotype(icd10_codes = "I84")
test_one_pheno

,I84
f.eid,
1000025,0
1000038,0
1000042,0
1000056,0
1000061,0
...,...
5873158,0
5873167,0
5873175,0


In [ ]:
get_phenotype(icd10_codes = "I84", samples = sample_ids)

,I84
f.eid,
5542886,0
5137974,0
3758348,0
1391800,0
3165331,0
...,...
3783812,0
2440570,0
1213317,0


In [ ]:
get_phenotype(icd10_codes = "I84", samples = sample_ids)[1].value_counts()

I84
0      344300
1       13343
dtype: int64

In [ ]:
get_phenotype(icd10_codes = "I84", samples = sample_ids)[1][:1000].value_counts()

I84
0      976
1       24
dtype: int64

In [ ]:
get_phenotype(icd10_codes = ["I84", "R07"])[1]

,I84,R07
f.eid,,
1000025,0,0
1000038,0,0
1000042,0,0
1000056,0,0
1000061,0,0
...,...,...
5873158,0,0
5873167,0,0
5873175,0,0


### Making phenotype balanced in case/control number

In [ ]:
test_phenotype_balanced =  get_phenotype(icd10_codes = ["I84", "R07"], samples=sample_ids, max_samples=10000, balance_pheno = "R07")
test_phenotype_balanced

,I84,R07
f.eid,,
5683283,0,1
1309216,0,1
4200400,0,1
4615802,0,1
1487885,0,1
...,...,...
4143614,0,0
2696228,0,0
1929723,0,0


Only one phenotype is balanced, the others are not. The balanced phenotype is sampled with replacement with `weights = 1/[class frequency]`

In [ ]:
test_phenotype_balanced.apply(pd.Series.value_counts)

,I84,R07
0,9494,4926
1,506,5074


---
## Get SNPs used in GWAS for trait

In [ ]:
#export

def get_GWAS_snps_for_trait(phenotype_code= "I84", chromosome:Union[int, List[int]] = 21, sort_val_cols_list: List[str] = ["pval"], ascending_bool_list: List[bool] = [True], id_only= True):
    chromosome = [chromosome] if not isinstance(chromosome, list) else chromosome
    chromosome_str = [f"{single_chromosome}:" for single_chromosome in chromosome]
    gwas_result_df = read_csv_compressed(get_GWAS_result_link(phenotype_code=phenotype_code))
    gwas_result_df = gwas_result_df.loc[gwas_result_df["variant"].str.startswith(tuple(chromosome_str)), :]
    gwas_result_df = gwas_result_df.reset_index(drop=True).reset_index().rename(columns = {"index":"position_rank"})
    gwas_result_df = gwas_result_df.sort_values(sort_val_cols_list, ascending = ascending_bool_list)
    variant_id_df = gwas_result_df["variant"].str.split(":",expand=True)
    variant_id_df["chr1_4"] =variant_id_df[[1,2,3]].apply("_".join, axis=1)
    variant_id_df[1] = variant_id_df[1].astype(int)
    gwas_result_df[["chr", "position", "major_allele"]] = variant_id_df[[0, 1, 2]]
    gwas_result_df["full_id"] =  variant_id_df[[0, "chr1_4"]].apply(":".join, axis=1)
    
    if id_only:
        return gwas_result_df["full_id"].values
    else:
        return gwas_result_df


In [ ]:
get_GWAS_snps_for_trait()

array(['21:46875774_C_T', '21:30968890_C_T', '21:43767602_A_G', ...,
       '21:46876064_G_C', '21:47773893_C_G', '21:47831648_C_T'],
      dtype=object)

In [ ]:
get_GWAS_snps_for_trait(id_only=False)

{21:         position_rank          variant minor_allele  minor_AF  \
 170594         170594  21:46875774:C:T            T  0.000007   
 84094           84094  21:30968890:C:T            T  0.000002   
 151278         151278  21:43767602:A:G            G  0.000002   
 88826           88826  21:31869421:T:G            G  0.000002   
 162176         162176  21:45587940:C:T            T  0.000003   
 ...               ...              ...          ...       ...   
 161896         161896  21:45545974:C:T            T  0.000000   
 165184         165184  21:46067181:G:C            C  0.000000   
 170601         170601  21:46876064:G:C            C  0.000000   
 176072         176072  21:47773893:C:G            G  0.000000   
 176489         176489  21:47831648:C:T            T  0.000000   
 
         expected_case_minor_AC  low_confidence_variant  n_complete_samples  \
 170594                0.172184                    True              361194   
 84094                 0.043356             

In [ ]:
test = get_GWAS_snps_for_trait(chromosome = [20, 21], id_only=False)
test

,position_rank,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,position,major_allele,full_id
466355,466355,21:46875774:C:T,T,6.791200e-06,0.172184,True,361194,4.905880,2.117650,0.667492,0.107762,6.19411,5.867920e-10,21,46875774,C,21:46875774_C_T
194304,194304,20:43850800:C:T,T,1.737160e-07,0.004404,True,361194,0.125490,0.086275,12.826500,2.294580,5.58991,2.273570e-08,20,43850800,C,20:43850800_C_T
149090,149090,20:33444611:C:T,T,1.644870e-06,0.041704,True,361194,1.188240,0.568627,2.003390,0.359746,5.56892,2.565050e-08,20,33444611,C,20:33444611_C_T
83527,83527,20:16253954:A:G,G,8.088640e-07,0.020508,True,361194,0.584314,0.533333,1.910680,0.354733,5.38626,7.198550e-08,20,16253954,A,20:16253954_A_G
183263,183263,20:41514559:A:G,G,3.257170e-08,0.000826,True,361194,0.023529,0.011765,88.836800,16.586000,5.35613,8.507440e-08,20,41514559,A,20:41514559_A_G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457657,457657,21:45545974:C:T,T,0.000000e+00,0.000000,True,361194,0.000000,0.000000,NaN,NaN,NaN,NaN,21,45545974,C,21:45545974_C_T
460945,460945,21:46067181:G:C,C,0.000000e+00,0.000000,True,361194,0.000000,0.000000,NaN,NaN,NaN,NaN,21,46067181,G,21:46067181_G_C
466362,466362,21:46876064:G:C,C,0.000000e+00,0.000000,True,361194,0.000000,0.000000,NaN,NaN,NaN,NaN,21,46876064,G,21:46876064_G_C
471833,471833,21:47773893:C:G,G,0.000000e+00,0.000000,True,361194,0.000000,0.000000,NaN,NaN,NaN,NaN,21,47773893,C,21:47773893_C_G


In [ ]:
assert sorted(test["chr"].unique()) == ['20', '21']